# Web Scraping from Altmetric

This code aims to scrape the latest information for all Tweets and summary page from Altmetric. The specific steps include:

1) Get the account handles (medialink) and headlines of all tweets, and count the numbers of our accounts. Since tweets are always without any subtitles, I didn't add this column here.

2) Put the medialinks and headlines into a new dataset "df", and add other columns from our original dataset "New_Altemtric_xlsx" which contains 'altmetric', '#article', 'author', 'mediatype', 'medialink', 'mediasource', 'mediaheadline', 'mediasubtitile'.

3) Fetch the Altmetric scores, citations, readers and demographic information, and add them into "df" according to existing paper links.

There is no raw data set. And after the above processing, this code will export a new google sheet "Altemtric_all_info.xlsx" which will also locate in the "Altmetric" folder.

In [1]:
# Import needed packages and connect to google drive
import pandas as pd
import requests
from bs4 import BeautifulSoup
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/_Fiverr/Altmetric

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/15NnxkAFgxXA4HcOuAuTgPTO4bHRb703i/_Fiverr/Altmetric


## Step 1
Fetch the account handles (medialink) and headlines of all tweets, and store them in lists.

In [2]:
# Fetch the links of all twitter pages for 7 papers on Altmetric
base_url2 = 'https://oxfordjournals.altmetric.com/details/72683542/twitter/page:' # Article 2 - Gangwisch
links2 = []
for i2 in range(1,4):
    page_url2 = f'{base_url2}{i2}'
    links2.append(page_url2)

base_url3 = 'https://science.altmetric.com/details/60552876/twitter/page:' # Article 3 - Lee
links3 = []
for i3 in range(1,6):
    page_url3 = f'{base_url3}{i3}'
    links3.append(page_url3)

base_url4 = 'https://jamanetwork.altmetric.com/details/64368646/twitter/page:' # Article 4 - Kim
links4 = []
for i4 in range(1,4):
    page_url4 = f'{base_url4}{i4}'
    links4.append(page_url4)

base_url5 = 'https://science.altmetric.com/details/69584866/twitter/page:' # Article 5 - Mina
links5 = []
for i5 in range(1,31):
    page_url5 = f'{base_url5}{i5}'
    links5.append(page_url5)

base_url6 = 'https://annals.altmetric.com/details/56459321/twitter/page:' # Article 6 - Hviid
links6 = []
for i6 in range(1,101):
    page_url6 = f'{base_url6}{i6}'
    links6.append(page_url6)

base_url7 = 'https://scienceadvances.altmetric.com/details/69530897/twitter/page:' # Article 7 - Maxwell
links7 = []
for i7 in range(1,6):
    page_url7 = f'{base_url7}{i7}'
    links7.append(page_url7)

base_url8 = 'https://nature.altmetric.com/details/63584063/twitter/page:' # Article 8 - Brezaghi
links8 = []
for i8 in range(1,4):
    page_url8 = f'{base_url8}{i8}'
    links8.append(page_url8)

# Combine them into a list 'links'
links = links2 + links3 +links4 + links5 + links6 + links7 + links8
links

['https://oxfordjournals.altmetric.com/details/72683542/twitter/page:1',
 'https://oxfordjournals.altmetric.com/details/72683542/twitter/page:2',
 'https://oxfordjournals.altmetric.com/details/72683542/twitter/page:3',
 'https://science.altmetric.com/details/60552876/twitter/page:1',
 'https://science.altmetric.com/details/60552876/twitter/page:2',
 'https://science.altmetric.com/details/60552876/twitter/page:3',
 'https://science.altmetric.com/details/60552876/twitter/page:4',
 'https://science.altmetric.com/details/60552876/twitter/page:5',
 'https://jamanetwork.altmetric.com/details/64368646/twitter/page:1',
 'https://jamanetwork.altmetric.com/details/64368646/twitter/page:2',
 'https://jamanetwork.altmetric.com/details/64368646/twitter/page:3',
 'https://science.altmetric.com/details/69584866/twitter/page:1',
 'https://science.altmetric.com/details/69584866/twitter/page:2',
 'https://science.altmetric.com/details/69584866/twitter/page:3',
 'https://science.altmetric.com/details/695

In [3]:
handles = []

for link in links:
    print(f'Medialinks for {link}:')
    text = requests.get(link).text
    soup = BeautifulSoup(text, 'html.parser')
    articles = soup.find_all('article', class_ = 'post twitter')

    # Get the account handles of all tweets
    for article in articles:
        author_handle = article.find('div', class_ = 'handle').text
        handles.append(author_handle)

    # Count the number of our accounts among them
    research_author_count = 0
    for article in articles:
        author_name = article.find('div', class_ = 'name').text
        if 'research' in author_name.lower() and 'finding' in author_name.lower(): # all of our Twitter accounts consist of "research" and "finding", so this line makes all the letters in account names lowercase and counts the accounts that contain these two words
            research_author_count += 1
    if research_author_count != 0:
        print(f"The number of our accounts: {research_author_count}")
    print('----------------------------------------------')

Medialinks for https://oxfordjournals.altmetric.com/details/72683542/twitter/page:1:
The number of our accounts: 99
----------------------------------------------
Medialinks for https://oxfordjournals.altmetric.com/details/72683542/twitter/page:2:
The number of our accounts: 6
----------------------------------------------
Medialinks for https://oxfordjournals.altmetric.com/details/72683542/twitter/page:3:
----------------------------------------------
Medialinks for https://science.altmetric.com/details/60552876/twitter/page:1:
The number of our accounts: 100
----------------------------------------------
Medialinks for https://science.altmetric.com/details/60552876/twitter/page:2:
The number of our accounts: 5
----------------------------------------------
Medialinks for https://science.altmetric.com/details/60552876/twitter/page:3:
----------------------------------------------
Medialinks for https://science.altmetric.com/details/60552876/twitter/page:4:
----------------------------

In [5]:
# Get the headlines of all tweets
headlines = []

for link in links:
    text = requests.get(link).text
    soup = BeautifulSoup(text, 'html.parser')
    articles = soup.find_all('article', class_ = 'post twitter')

    for article in articles:
        headline = article.find('p', class_ = 'summary').text
        headlines.append(headline)

In [6]:
# Check the numbers of headlines and handles
print(f'There are {len(handles)} tweets.')
print(f'There are {len(headlines)} headlines.')

There are 14573 tweets.
There are 14573 headlines.


## Step 2
Put the above info into a new data frame "df", and add the other columns and values.

In [7]:
data = {
    'medialink': handles,
    'mediaheadline': headlines,
}
df = pd.DataFrame(data)
df

,medialink,mediaheadline
0,@tolucky39,参考文献\nHigh glycemic index and glycemic load di...
1,@researchfindin,New study finds shocking link between High GI ...
2,@researchfindstd,Your Diet Could be the Culprit Behind Insomnia...
3,@research_finds_,New Study Finds Probable Link Between Diet and...
4,@researchfindin,Is Your Diet Triggering Insomnia? https://t.co...
...,...,...
14568,@Zacarias15D,"Elephants, megagarderners of African rainfores..."
14569,@geomatlab,#Climate NPG: Carbon stocks in central African...
14570,@savieira08,Acaba de sair nosso artigo sobre o papel dos e...
14571,@tlyadeen,RT @NatureGeosci: NGeo: Decline of elephant po...


In [8]:
# Add a conlumn that includes "altmetric" link
df.loc[0:218, 'altmetric'] = 'https://oxfordjournals.altmetric.com/details/72683542' # Article 2 - Gangwisch
df.loc[219:673, 'altmetric'] = 'https://science.altmetric.com/details/60552876' # Article 3 - Lee
df.loc[674:885, 'altmetric'] = 'https://jamanetwork.altmetric.com/details/64368646' # Article 4 - Kim
df.loc[886:3858, 'altmetric'] = 'https://science.altmetric.com/details/69584866' # Article 5 - Mina
df.loc[3859:13858, 'altmetric'] = 'https://annals.altmetric.com/details/56459321' # Article 6 - Hviid
df.loc[13859:14312, 'altmetric'] = 'https://scienceadvances.altmetric.com/details/69530897' # Article 7 - Maxwell
df.loc[14313:14572, 'altmetric'] = 'https://nature.altmetric.com/details/63584063' # Article 8 - Brezaghi

# Add a conlumn "mediatype"
df['mediatype'] = 'Tweet'

# Add a conlumn "mediasource"
df['mediasource'] = 'Twitter'

# Add a conlumn "mediasubtitile"
df['mediasubtitile'] = ''

# Add a conlumn "Author"
df.loc[df['altmetric'] == 'https://oxfordjournals.altmetric.com/details/72683542', 'author'] = 'Gangwisch et al.'
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/60552876', 'author'] = 'Lee et al.'
df.loc[df['altmetric'] == 'https://jamanetwork.altmetric.com/details/64368646', 'author'] = 'Kim et al.'
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/69584866', 'author'] = 'Mina et al.'
df.loc[df['altmetric'] == 'https://annals.altmetric.com/details/56459321', 'author'] = 'Hviid et al.'
df.loc[df['altmetric'] == 'https://scienceadvances.altmetric.com/details/69530897', 'author'] = 'Maxwell et al.'
df.loc[df['altmetric'] == 'https://nature.altmetric.com/details/63584063', 'author'] = 'Berzaghi et al.'

# Add a conlumn "#Article"
df.loc[df['altmetric'] == 'https://oxfordjournals.altmetric.com/details/72683542', '#article'] = 'Article 2'
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/60552876', '#article'] = 'Article 3'
df.loc[df['altmetric'] == 'https://jamanetwork.altmetric.com/details/64368646', '#article'] = 'Article 4'
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/69584866', '#article'] = 'Article 5'
df.loc[df['altmetric'] == 'https://annals.altmetric.com/details/56459321', '#article'] = 'Article 6'
df.loc[df['altmetric'] == 'https://scienceadvances.altmetric.com/details/69530897', '#article'] = 'Article 7'
df.loc[df['altmetric'] == 'https://nature.altmetric.com/details/63584063', '#article'] = 'Article 8'

df = df[['altmetric', '#article', 'author', 'mediatype', 'medialink', 'mediasource', 'mediaheadline', 'mediasubtitile']]
df


,altmetric,#article,author,mediatype,medialink,mediasource,mediaheadline,mediasubtitile
0,https://oxfordjournals.altmetric.com/details/7...,Article 2,Gangwisch et al.,Tweet,@tolucky39,Twitter,参考文献\nHigh glycemic index and glycemic load di...,
1,https://oxfordjournals.altmetric.com/details/7...,Article 2,Gangwisch et al.,Tweet,@researchfindin,Twitter,New study finds shocking link between High GI ...,
2,https://oxfordjournals.altmetric.com/details/7...,Article 2,Gangwisch et al.,Tweet,@researchfindstd,Twitter,Your Diet Could be the Culprit Behind Insomnia...,
3,https://oxfordjournals.altmetric.com/details/7...,Article 2,Gangwisch et al.,Tweet,@research_finds_,Twitter,New Study Finds Probable Link Between Diet and...,
4,https://oxfordjournals.altmetric.com/details/7...,Article 2,Gangwisch et al.,Tweet,@researchfindin,Twitter,Is Your Diet Triggering Insomnia? https://t.co...,
...,...,...,...,...,...,...,...,...
14568,https://nature.altmetric.com/details/63584063,Article 8,Berzaghi et al.,Tweet,@Zacarias15D,Twitter,"Elephants, megagarderners of African rainfores...",
14569,https://nature.altmetric.com/details/63584063,Article 8,Berzaghi et al.,Tweet,@geomatlab,Twitter,#Climate NPG: Carbon stocks in central African...,
14570,https://nature.altmetric.com/details/63584063,Article 8,Berzaghi et al.,Tweet,@savieira08,Twitter,Acaba de sair nosso artigo sobre o papel dos e...,
14571,https://nature.altmetric.com/details/63584063,Article 8,Berzaghi et al.,Tweet,@tlyadeen,Twitter,RT @NatureGeosci: NGeo: Decline of elephant po...,


In [9]:
# Get the summary information of all 7 papers from Altmetric
links = [
    'https://oxfordjournals.altmetric.com/details/72683542', # Article 2 - Gangwisch
    'https://science.altmetric.com/details/60552876', # Article 3 - Lee
    'https://jamanetwork.altmetric.com/details/64368646', # Article 4 - Kim
    'https://science.altmetric.com/details/69584866', # Article 5 - Mina
    'https://annals.altmetric.com/details/56459321', # Article 6 - Hviid
    'https://scienceadvances.altmetric.com/details/69530897', # Article 7 - Maxwell
    'https://nature.altmetric.com/details/63584063', # Article 8 - Brezaghi
]
scores = []
citations = []
readers = []
count = []
percent = []

for index, link in enumerate (links, start = 2):
    text = requests.get(link).text
    soup = BeautifulSoup(text, 'html.parser')

    # Get the Altmetric score
    score = soup.find('div', class_ = 'altmetric-badge')
    s = score['style']
    s_start = s[s.find('score=')+6:]
    s_end = s_start[:s_start.find('&')]
    scores.append(s_end)
    print(f'Score for Article {index} is: {s_end}')

    # Get citations
    citation = soup.find('div', class_ = 'scholarly-citation-counts-wrapper')
    c = citation.find('strong').text
    citations.append(c)
    print(f'Citation for Article {index} is: {c}')

    # Get readers
    reader = soup.find('div', class_ = 'reader-counts-wrapper')
    r = reader.find('strong').text
    readers.append(r)
    print(f'Readers for Article {index} is: {r}')

    # Get the demographics infomation
    table = soup.find('div', class_ = 'table-wrapper users')
    for td in list(list(table.children)[3].children)[3::2]:
        caption = td.find('td')
        nums = [i.text for i in td.find_all('td', class_ = 'num')]
        count.append(nums[0])
        percent.append(nums[1])
        print(caption.text)
        print(nums)
    print('\n----------------------------------------------\n')
print(count, percent)

Score for Article 2 is: 533
Citation for Article 2 is: 51
Readers for Article 2 is: 156
Members of the public
['87', '75%']
Practitioners (doctors, other healthcare professionals)
['17', '15%']
Scientists
['12', '10%']

----------------------------------------------

Score for Article 3 is: 507
Citation for Article 3 is: 176
Readers for Article 3 is: 245
Members of the public
['241', '70%']
Scientists
['64', '19%']
Practitioners (doctors, other healthcare professionals)
['28', '8%']
Science communicators (journalists, bloggers, editors)
['9', '3%']

----------------------------------------------

Score for Article 4 is: 506
Citation for Article 4 is: 31
Readers for Article 4 is: 59
Members of the public
['74', '76%']
Practitioners (doctors, other healthcare professionals)
['12', '12%']
Scientists
['9', '9%']
Science communicators (journalists, bloggers, editors)
['2', '2%']

----------------------------------------------

Score for Article 5 is: 3665
Citation for Article 5 is: 265
Read

## Step 3
Add the summary information to "df".

In [11]:
# Add a conlumn "Score"
df.loc[df['altmetric'] == 'https://oxfordjournals.altmetric.com/details/72683542', 'score'] = scores[0]
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/60552876', 'score'] = scores[1]
df.loc[df['altmetric'] == 'https://jamanetwork.altmetric.com/details/64368646', 'score'] = scores[2]
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/69584866', 'score'] = scores[3]
df.loc[df['altmetric'] == 'https://annals.altmetric.com/details/56459321', 'score'] = scores[4]
df.loc[df['altmetric'] == 'https://scienceadvances.altmetric.com/details/69530897', 'score'] = scores[5]
df.loc[df['altmetric'] == 'https://nature.altmetric.com/details/63584063', 'score'] = scores[6]

# Add a conlumn "citations"
df.loc[df['altmetric'] == 'https://oxfordjournals.altmetric.com/details/72683542', 'citations'] = citations[0]
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/60552876', 'citations'] = citations[1]
df.loc[df['altmetric'] == 'https://jamanetwork.altmetric.com/details/64368646', 'citations'] = citations[2]
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/69584866', 'citations'] = citations[3]
df.loc[df['altmetric'] == 'https://annals.altmetric.com/details/56459321', 'citations'] = citations[4]
df.loc[df['altmetric'] == 'https://scienceadvances.altmetric.com/details/69530897', 'citations'] = citations[5]
df.loc[df['altmetric'] == 'https://nature.altmetric.com/details/63584063', 'citations'] = citations[6]

# Add a conlumn "readers"
df.loc[df['altmetric'] == 'https://oxfordjournals.altmetric.com/details/72683542', 'readers'] = readers[0]
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/60552876', 'readers'] = readers[1]
df.loc[df['altmetric'] == 'https://jamanetwork.altmetric.com/details/64368646', 'readers'] = readers[2]
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/69584866', 'readers'] = readers[3]
df.loc[df['altmetric'] == 'https://annals.altmetric.com/details/56459321', 'readers'] = readers[4]
df.loc[df['altmetric'] == 'https://scienceadvances.altmetric.com/details/69530897', 'readers'] = readers[5]
df.loc[df['altmetric'] == 'https://nature.altmetric.com/details/63584063', 'readers'] = readers[6]

# Add a conlumn "demo_public_count"
df.loc[df['altmetric'] == 'https://oxfordjournals.altmetric.com/details/72683542', 'demo_public_count'] = count[0]
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/60552876', 'demo_public_count'] = count[3]
df.loc[df['altmetric'] == 'https://jamanetwork.altmetric.com/details/64368646', 'demo_public_count'] = count[7]
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/69584866', 'demo_public_count'] = count[11]
df.loc[df['altmetric'] == 'https://annals.altmetric.com/details/56459321', 'demo_public_count'] = count[15]
df.loc[df['altmetric'] == 'https://scienceadvances.altmetric.com/details/69530897', 'demo_public_count'] = count[20]
df.loc[df['altmetric'] == 'https://nature.altmetric.com/details/63584063', 'demo_public_count'] = count[24]

# Add a conlumn "demo_public_perc"
df.loc[df['altmetric'] == 'https://oxfordjournals.altmetric.com/details/72683542', 'demo_public_perc'] = percent[0]
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/60552876', 'demo_public_perc'] = percent[3]
df.loc[df['altmetric'] == 'https://jamanetwork.altmetric.com/details/64368646', 'demo_public_perc'] = percent[7]
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/69584866', 'demo_public_perc'] = percent[11]
df.loc[df['altmetric'] == 'https://annals.altmetric.com/details/56459321', 'demo_public_perc'] = percent[15]
df.loc[df['altmetric'] == 'https://scienceadvances.altmetric.com/details/69530897', 'demo_public_perc'] = percent[20]
df.loc[df['altmetric'] == 'https://nature.altmetric.com/details/63584063', 'demo_public_perc'] = percent[24]

# Add a conlumn "demo_scientist_count"
df.loc[df['altmetric'] == 'https://oxfordjournals.altmetric.com/details/72683542', 'demo_scientist_count'] = count[2]
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/60552876', 'demo_scientist_count'] = count[4]
df.loc[df['altmetric'] == 'https://jamanetwork.altmetric.com/details/64368646', 'demo_scientist_count'] = count[9]
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/69584866', 'demo_scientist_count'] = count[12]
df.loc[df['altmetric'] == 'https://annals.altmetric.com/details/56459321', 'demo_scientist_count'] = count[16]
df.loc[df['altmetric'] == 'https://scienceadvances.altmetric.com/details/69530897', 'demo_scientist_count'] = count[21]
df.loc[df['altmetric'] == 'https://nature.altmetric.com/details/63584063', 'demo_scientist_count'] = count[25]

# Add a conlumn "demo_scientist_perc"
df.loc[df['altmetric'] == 'https://oxfordjournals.altmetric.com/details/72683542', 'demo_scientist_perc'] = percent[2]
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/60552876', 'demo_scientist_perc'] = percent[4]
df.loc[df['altmetric'] == 'https://jamanetwork.altmetric.com/details/64368646', 'demo_scientist_perc'] = percent[9]
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/69584866', 'demo_scientist_perc'] = percent[12]
df.loc[df['altmetric'] == 'https://annals.altmetric.com/details/56459321', 'demo_scientist_perc'] = percent[16]
df.loc[df['altmetric'] == 'https://scienceadvances.altmetric.com/details/69530897', 'demo_scientist_perc'] = percent[21]
df.loc[df['altmetric'] == 'https://nature.altmetric.com/details/63584063', 'demo_scientist_perc'] = percent[25]

# Add a conlumn "demo_practitioners_count"
df.loc[df['altmetric'] == 'https://oxfordjournals.altmetric.com/details/72683542', 'demo_practitioners_count'] = count[1]
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/60552876', 'demo_practitioners_count'] = count[5]
df.loc[df['altmetric'] == 'https://jamanetwork.altmetric.com/details/64368646', 'demo_practitioners_count'] = count[8]
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/69584866', 'demo_practitioners_count'] = count[13]
df.loc[df['altmetric'] == 'https://annals.altmetric.com/details/56459321', 'demo_practitioners_count'] = count[17]
df.loc[df['altmetric'] == 'https://scienceadvances.altmetric.com/details/69530897', 'demo_practitioners_count'] = count[23]
df.loc[df['altmetric'] == 'https://nature.altmetric.com/details/63584063', 'demo_practitioners_count'] = 0

# Add a conlumn "demo_practitioners_perc"
df.loc[df['altmetric'] == 'https://oxfordjournals.altmetric.com/details/72683542', 'demo_practitioners_perc'] = percent[1]
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/60552876', 'demo_practitioners_perc'] = percent[5]
df.loc[df['altmetric'] == 'https://jamanetwork.altmetric.com/details/64368646', 'demo_practitioners_perc'] = percent[8]
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/69584866', 'demo_practitioners_perc'] = percent[13]
df.loc[df['altmetric'] == 'https://annals.altmetric.com/details/56459321', 'demo_practitioners_perc'] = percent[17]
df.loc[df['altmetric'] == 'https://scienceadvances.altmetric.com/details/69530897', 'demo_practitioners_perc'] = percent[23]
df.loc[df['altmetric'] == 'https://nature.altmetric.com/details/63584063', 'demo_practitioners_perc'] = 0

# Add a conlumn "demo_science_communicators_count"
df.loc[df['altmetric'] == 'https://oxfordjournals.altmetric.com/details/72683542', 'demo_science_communicators_count'] = 0
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/60552876', 'demo_science_communicators_count'] = count[6]
df.loc[df['altmetric'] == 'https://jamanetwork.altmetric.com/details/64368646', 'demo_science_communicators_count'] = count[10]
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/69584866', 'demo_science_communicators_count'] = count[14]
df.loc[df['altmetric'] == 'https://annals.altmetric.com/details/56459321', 'demo_science_communicators_count'] = count[18]
df.loc[df['altmetric'] == 'https://scienceadvances.altmetric.com/details/69530897', 'demo_science_communicators_count'] = count[22]
df.loc[df['altmetric'] == 'https://nature.altmetric.com/details/63584063', 'demo_science_communicators_count'] = count[26]

# Add a conlumn "demo_science_communicators_perc"
df.loc[df['altmetric'] == 'https://oxfordjournals.altmetric.com/details/72683542', 'demo_science_communicators_perc'] = 0
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/60552876', 'demo_science_communicators_perc'] = percent[6]
df.loc[df['altmetric'] == 'https://jamanetwork.altmetric.com/details/64368646', 'demo_science_communicators_perc'] = percent[10]
df.loc[df['altmetric'] == 'https://science.altmetric.com/details/69584866', 'demo_science_communicators_perc'] = percent[14]
df.loc[df['altmetric'] == 'https://annals.altmetric.com/details/56459321', 'demo_science_communicators_perc'] = percent[18]
df.loc[df['altmetric'] == 'https://scienceadvances.altmetric.com/details/69530897', 'demo_science_communicators_perc'] = percent[22]
df.loc[df['altmetric'] == 'https://nature.altmetric.com/details/63584063', 'demo_science_communicators_perc'] = percent[26]

df

,altmetric,#article,author,mediatype,medialink,mediasource,mediaheadline,mediasubtitile,score,citations,readers,demo_public_count,demo_public_perc,demo_scientist_count,demo_scientist_perc,demo_practitioners_count,demo_practitioners_perc,demo_science_communicators_count,demo_science_communicators_perc
0,https://oxfordjournals.altmetric.com/details/7...,Article 2,Gangwisch et al.,Tweet,@tolucky39,Twitter,参考文献\nHigh glycemic index and glycemic load di...,,533,51,156,87,75%,12,10%,17,15%,0,0
1,https://oxfordjournals.altmetric.com/details/7...,Article 2,Gangwisch et al.,Tweet,@researchfindin,Twitter,New study finds shocking link between High GI ...,,533,51,156,87,75%,12,10%,17,15%,0,0
2,https://oxfordjournals.altmetric.com/details/7...,Article 2,Gangwisch et al.,Tweet,@researchfindstd,Twitter,Your Diet Could be the Culprit Behind Insomnia...,,533,51,156,87,75%,12,10%,17,15%,0,0
3,https://oxfordjournals.altmetric.com/details/7...,Article 2,Gangwisch et al.,Tweet,@research_finds_,Twitter,New Study Finds Probable Link Between Diet and...,,533,51,156,87,75%,12,10%,17,15%,0,0
4,https://oxfordjournals.altmetric.com/details/7...,Article 2,Gangwisch et al.,Tweet,@researchfindin,Twitter,Is Your Diet Triggering Insomnia? https://t.co...,,533,51,156,87,75%,12,10%,17,15%,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14568,https://nature.altmetric.com/details/63584063,Article 8,Berzaghi et al.,Tweet,@Zacarias15D,Twitter,"Elephants, megagarderners of African rainfores...",,1340,61,211,137,87%,14,9%,0,0,6,4%
14569,https://nature.altmetric.com/details/63584063,Article 8,Berzaghi et al.,Tweet,@geomatlab,Twitter,#Climate NPG: Carbon stocks in central African...,,1340,61,211,137,87%,14,9%,0,0,6,4%
14570,https://nature.altmetric.com/details/63584063,Article 8,Berzaghi et al.,Tweet,@savieira08,Twitter,Acaba de sair nosso artigo sobre o papel dos e...,,1340,61,211,137,87%,14,9%,0,0,6,4%
14571,https://nature.altmetric.com/details/63584063,Article 8,Berzaghi et al.,Tweet,@tlyadeen,Twitter,RT @NatureGeosci: NGeo: Decline of elephant po...,,1340,61,211,137,87%,14,9%,0,0,6,4%


In [ ]:
# Export a new sheet which will locate in our "Altmetric" folder
df.to_excel('Altemtric_all_info.xlsx', index=False)